In [1]:
import setGPU
import tensorflow as tf

setGPU: Setting GPU to: 0


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
img_width, img_height = 299, 299
BATCH_SIZE = 128
EPOCHS = 2
IMG_SHAPE = (img_width, img_height, 3)
_NUM_CLASSES = 76

In [4]:
train_data_path = '../../../data/movie_titles/data/train/'
valid_data_path = '../../../data/movie_titles/data/validation/'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_path, target_size=(img_width, img_height), 
                                                    batch_size=BATCH_SIZE, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_data_path, target_size=(img_width, img_height), 
                                                    batch_size=BATCH_SIZE, class_mode='categorical')

Found 567680 images belonging to 76 classes.
Found 141881 images belonging to 76 classes.


In [5]:
conv_base = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

head_model = conv_base.output
head_model = tf.keras.layers.GlobalAveragePooling2D()(head_model)
head_model = tf.keras.layers.Flatten(name='flatten')(head_model)
head_model = tf.keras.layers.Dense(1024, activation='relu', name='dense1')(head_model)
head_model = tf.keras.layers.Dense(512, activation='relu', name='dense2')(head_model)
head_model = tf.keras.layers.Dense(256, activation='relu', name='dense3')(head_model)
head_model = tf.keras.layers.Dense(128, activation='relu', name='dense4')(head_model)
head_model = tf.keras.layers.Dropout(0.5)(head_model)
head_model = tf.keras.layers.Dense(_NUM_CLASSES, activation='softmax', name='final_predictions')(head_model)

model = tf.keras.Model(inputs=conv_base.inputs, outputs=head_model)
#model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
#trigger training
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#callbacks
#lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
file_path = "../../../model/120919/weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = '../../model/120919/logs/', histogram_freq=0, write_graph=True)

#callback_list = [lr_callback, es_callback, checkpoint, tb_callback]
callback_list = [es_callback, checkpoint, tb_callback]

train_steps = train_generator.n//BATCH_SIZE
valid_steps = valid_generator.n//BATCH_SIZE
model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=EPOCHS, workers=16, validation_data=valid_generator, 
                    validation_steps=valid_steps, use_multiprocessing=True, callbacks=callback_list)

model.save_weights('../../../model/120919/final_model.h5')

Epoch 1/2
  42/4435 [..............................] - ETA: 49:12 - loss: 4.1808 - acc: 0.0372